In [10]:
import json
import os
import subprocess
from repo_path_mapping import repos


def sync_repo_readmes(metadata_path, dry_run=True):
  """
  Checks each repository for a README. If missing, creates one with a title 
  and description and performs git add, commit, and push.
  """
  # Load the repository metadata
  try:
    with open(metadata_path, 'r', encoding='utf-8') as f:
      metadata_list = json.load(f)
  except FileNotFoundError:
    print(f"Error: Metadata file not found at {metadata_path}")
    return

  # Convert metadata list to a dictionary for faster lookup
  repo_metadata = {repo['name']: repo for repo in metadata_list}

  print(f"{'--- DRY RUN MODE (No changes will be made) ---' if dry_run else '--- LIVE EXECUTION MODE ---'}\n")

  for repo_name, local_path in repos.items():
    # Verify the local path exists
    if not os.path.exists(local_path):
      print(f"Skipping: Path does not exist for {repo_name} -> {local_path}")
      continue

    # Get specific metadata for this repository
    data = repo_metadata.get(repo_name)
    if not data:
      print(f"Skipping: No metadata found in JSON for repository '{repo_name}'")
      continue

    readme_path = os.path.join(local_path, "README.md")

    # Only proceed if README doesn't exist
    if not os.path.exists(readme_path):
      title = data.get('name', repo_name)
      description = data.get('description', 'No description available.')
      content = f"# {title}\n\n{description}\n"

      print(f"Action: Creating README for {repo_name}")

      if not dry_run:
        with open(readme_path, 'w', encoding='utf-8') as f:
          f.write(content)

      # Git operations
      try:
        def run_git(args):
          if dry_run:
            print(f"  [Dry Run] git {' '.join(args)}")
          else:
            subprocess.run(["git"] + args, cwd=local_path, check=True, capture_output=True)

        run_git(["add", "README.md"])
        run_git(["commit", "-m", "added README"])
        run_git(["push"])

        if not dry_run:
          print(f"Successfully updated {repo_name} on GitHub.")

      except subprocess.CalledProcessError as e:
        error_msg = e.stderr.decode().strip()
        print(f"Error updating {repo_name}: {error_msg}")
    else:
      print(f"Skipping: README already exists for {repo_name}")

In [11]:
sync_repo_readmes('repos_k26rahul.json', dry_run=True)

--- DRY RUN MODE (No changes will be made) ---

Skipping: README already exists for ai-agent-hackathon
Skipping: README already exists for cricket-hackathon-2023
Skipping: README already exists for space-hackathon
Skipping: README already exists for microsoft-ai-odyssey
Skipping: README already exists for studymate
Skipping: README already exists for rythmfit-hack-o-pitch-2.0
Skipping: README already exists for violence-detection-hackathon
Skipping: README already exists for pixelmind-ai-hackathon
Skipping: README already exists for ai-rpg-chat
Skipping: README already exists for auth-system-flask
Skipping: README already exists for bs-grade-calculator
Skipping: README already exists for event-emitter-lib
Skipping: README already exists for expense-tracker-react
Skipping: README already exists for html-projects
Skipping: README already exists for inventory-management-system
Skipping: README already exists for js-projects
Skipping: README already exists for js-projects-2
Skipping: READM